This is a new script generated using the reference link - https://fenicsproject.discourse.group/t/transitioning-from-mesh-xml-to-mesh-xdmf-from-dolfin-convert-to-meshio/412/3

In [2]:
import meshio
import json
import os

In [3]:
# Define input and output paths
MESH_INPUT = "mesh/cantilever_marked.med"
MESH_OUTPUT_DIR = "mesh/output"
MARKERS_OUTPUT_FILE = os.path.join(MESH_OUTPUT_DIR, "markers.json")

# Create output directory if it doesn't exist
os.makedirs(MESH_OUTPUT_DIR, exist_ok=True)

In [4]:
msh = meshio.read(MESH_INPUT)

# Extract mesh components
points = msh.points
cell = msh.cells
cell_data = msh.cell_data

# Helper: pick a tag key that exists (MED/Gmsh pipelines vary)
def pick_tag_key(cdict, ctype):
    for k in ("cell_tags", "gmsh:physical", "medit:ref"):
        if ctype in cdict and k in cdict[ctype]:
            return k
    return None

# --- Optional: write a plain volume mesh (no tags) like your original 'mesh.xdmf'
if "tetra" in dict(cell):
    tetra_cells = dict(cell)["tetra"]
    meshio.write(os.path.join(MESH_OUTPUT_DIR, "mesh.xdmf"), meshio.Mesh(
        points=points[:, :],
        cells={"tetra": tetra_cells},
    ))

# Check for triangle cells and write surface mesh with cell data
if "triangle" in dict(cell):
    triangle_cells = dict(cell)["triangle"]
    triangle_tags = cell_data["triangle"]["cell_tags"]
    meshio.write(os.path.join(MESH_OUTPUT_DIR, "surface.xdmf"), meshio.Mesh(
        points=points[:, :],
        cells={"triangle": triangle_cells},
        cell_data={"triangle": {"marker": triangle_tags}}
    ))

# Check for tetra cells and write solid mesh with cell data
if "tetra" in dict(cell):
    tetra_cells = dict(cell)["tetra"]
    tetra_tags = cell_data["tetra"]["cell_tags"]
    meshio.write(os.path.join(MESH_OUTPUT_DIR, "solid.xdmf"), meshio.Mesh(
        points=points[:, :],
        cells={"tetra": tetra_cells},
        cell_data={"tetra": {"marker": tetra_tags}}
    ))

# --- Combined mesh with tetra and triangle cells ---
combined_cells = {}
combined_cell_data = {}

if "tetra" in dict(cell):
    combined_cells["tetra"] = dict(cell)["tetra"]
    if "tetra" in cell_data and "cell_tags" in cell_data["tetra"]:
        combined_cell_data["tetra"] = {"marker": cell_data["tetra"]["cell_tags"]}

if "triangle" in dict(cell):
    combined_cells["triangle"] = dict(cell)["triangle"]
    if "triangle" in cell_data and "cell_tags" in cell_data["triangle"]:
        combined_cell_data["triangle"] = {"marker": cell_data["triangle"]["cell_tags"]}

if combined_cells:
    meshio.write(
        os.path.join(MESH_OUTPUT_DIR, "combined.xdmf"),
        meshio.Mesh(
            points=points,
            cells=combined_cells,
            cell_data=combined_cell_data
        )
    )

print(f"Conversion complete. Mesh and cell function saved in {MESH_OUTPUT_DIR}.")

PermissionError: [Errno 13] Unable to synchronously create file (unable to open file: name = 'mesh/output/mesh.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 242)

In [ ]:
# Read MED mesh
mesh = msh

points, cell, cell_data, field_data = mesh.points, mesh.cells, mesh.cell_data, mesh.field_data
cell_tags = mesh.cell_tags  # group → ids mapping

# Mapping of cell types to geometric dimensions
cell_type_dim = {
    "triangle": 2,
    "tetra": 3
}

# --- Build marker mapping (only for surface=2D and solid=3D) ---
marker_map = {}
for tag, names in cell_tags.items():
    tag_int = int(tag)
    found_dim = -1
    for cell_type, dim in cell_type_dim.items():
        tag_array = cell_data.get(cell_type, {}).get("cell_tags", None)
        if tag_array is not None and tag_int in tag_array:
            found_dim = dim
            break
    if found_dim in (2, 3):  # keep only surface and solid
        for name in names:
            marker_map[name] = {
                "id": tag_int,
                "dim": found_dim,
                "name": name
            }

# --- Separate surface vs solid ---
surface_markers = {n: info for n, info in marker_map.items() if info["dim"] == 2}
solid_markers   = {n: info for n, info in marker_map.items() if info["dim"] == 3}

# --- Save both into one JSON ---
all_markers = {"surface": surface_markers, "solid": solid_markers}
with open(MARKERS_OUTPUT_FILE, "w") as f:
    json.dump(all_markers, f, indent=4)

# --- Print result ---
print(f"Markers saved to {MARKERS_OUTPUT_FILE}:")
print(json.dumps(all_markers, indent=4))